In [1]:
import pandas as pd
import ast
import re
from datetime import datetime

%load_ext autoreload
%autoreload 2

*   ETL - USER REVIEWS 

In [2]:
user_reviews_ruta = 'data/australian_user_reviews.json'

# Lista para almacenar los diccionarios JSON de cada línea
data_list = []

# Abrir el archivo y procesar cada línea
with open(user_reviews_ruta, 'r', encoding='utf-8') as file:
    for line in file:
        try:
            # Usar ast.literal_eval para convertir la línea en un diccionario
            json_data = ast.literal_eval(line)
            data_list.append(json_data)
        except ValueError as e:
            print(f"Error en la línea: {line}")
            continue

# Crear un DataFrame a partir de la lista de diccionarios
df_reviews = pd.DataFrame(data_list)

In [3]:
df_reviews

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."
...,...,...,...
25794,76561198306599751,http://steamcommunity.com/profiles/76561198306...,"[{'funny': '', 'posted': 'Posted May 31.', 'la..."
25795,Ghoustik,http://steamcommunity.com/id/Ghoustik,"[{'funny': '', 'posted': 'Posted June 17.', 'l..."
25796,76561198310819422,http://steamcommunity.com/profiles/76561198310...,"[{'funny': '1 person found this review funny',..."
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"[{'funny': '', 'posted': 'Posted July 21.', 'l..."


Primero se revisarán los duplicados basados en user_id y luego más adelante <br>
se tomará el tiempo de hacer las modificaciones para la columna reviews

In [4]:
# Se buscan los duplicados que puedan haber por cada user_id
duplicated_rows = df_reviews[df_reviews.duplicated(subset='user_id', keep=False)]
duplicated_rows_sorted = duplicated_rows.sort_values(by='user_id')
duplicated_rows_sorted

,user_id,user_url,reviews
12888,05041129,http://steamcommunity.com/id/05041129,"[{'funny': '', 'posted': 'Posted May 18, 2015...."
5250,05041129,http://steamcommunity.com/id/05041129,"[{'funny': '', 'posted': 'Posted May 18, 2015...."
3133,111222333444555666888,http://steamcommunity.com/id/11122233344455566...,"[{'funny': '', 'posted': 'Posted December 22, ..."
3134,111222333444555666888,http://steamcommunity.com/id/11122233344455566...,"[{'funny': '', 'posted': 'Posted December 22, ..."
4139,29123,http://steamcommunity.com/id/29123,"[{'funny': '', 'posted': 'Posted March 26.', '..."
...,...,...,...
2721,xXAussieRockXx,http://steamcommunity.com/id/xXAussieRockXx,"[{'funny': '', 'posted': 'Posted July 17, 2015..."
2680,yolofaceguy,http://steamcommunity.com/id/yolofaceguy,"[{'funny': '', 'posted': 'Posted October 31, 2..."
17916,yolofaceguy,http://steamcommunity.com/id/yolofaceguy,"[{'funny': '', 'posted': 'Posted October 31, 2..."
5855,zeroblade,http://steamcommunity.com/id/zeroblade,"[{'funny': '', 'posted': 'Posted November 30, ..."


In [5]:
# Se procede a eliminar duplicados dejando el primer review
df_reviews = df_reviews.drop_duplicates(subset='user_id', keep='first')

In [6]:
# Se vuelve a revisar la cantidad de duplicados
duplicated_rows = df_reviews[df_reviews.duplicated(subset='user_id', keep=False)]
duplicated_rows_sorted = duplicated_rows.sort_values(by='user_id')
duplicated_rows_sorted

,user_id,user_url,reviews


Ahora, observando el df a primera vista salta la columna de reviews, primero se va a aplicar <br>
la funcion json_normalize para "aplanar" la estructura anidada del json <br>
y tenerlo en un data frame tabular

In [7]:
# Se transforma el json a un formato tabular en columnas
df_reviews_norm = pd.json_normalize(df_reviews['reviews'])
df_reviews_norm.head()

,0,1,2,3,4,5,6,7,8,9
0,"{'funny': '', 'posted': 'Posted November 5, 20...","{'funny': '', 'posted': 'Posted July 15, 2011....","{'funny': '', 'posted': 'Posted April 21, 2011...",None,None,None,None,None,None,None
1,"{'funny': '', 'posted': 'Posted June 24, 2014....","{'funny': '', 'posted': 'Posted September 8, 2...","{'funny': '', 'posted': 'Posted November 29, 2...",None,None,None,None,None,None,None
2,"{'funny': '', 'posted': 'Posted February 3.', ...","{'funny': '', 'posted': 'Posted December 4, 20...","{'funny': '', 'posted': 'Posted November 3, 20...","{'funny': '', 'posted': 'Posted October 15, 20...","{'funny': '', 'posted': 'Posted October 15, 20...","{'funny': '', 'posted': 'Posted October 15, 20...",None,None,None,None
3,"{'funny': '', 'posted': 'Posted October 14, 20...","{'funny': '', 'posted': 'Posted July 28, 2012....","{'funny': '', 'posted': 'Posted June 2, 2012.'...","{'funny': '', 'posted': 'Posted June 29, 2014....","{'funny': '', 'posted': 'Posted November 22, 2...","{'funny': '', 'posted': 'Posted February 23, 2...",None,None,None,None
4,"{'funny': '3 people found this review funny', ...","{'funny': '1 person found this review funny', ...","{'funny': '2 people found this review funny', ...","{'funny': '', 'posted': 'Posted July 11, 2013....",None,None,None,None,None,None


Como observamos, ya no aparecen las columnas de id y url que se tenian <br>
en df_reviews, por lo cual se concatenan a este nuevo df

In [8]:
# Se concatenan las columnas que se perdieron
df_reviews_norm = pd.concat([df_reviews[['user_id', 'user_url']], df_reviews_norm], axis=1)
df_reviews_norm.head()

,user_id,user_url,0,1,2,3,4,5,6,7,8,9
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted November 5, 20...","{'funny': '', 'posted': 'Posted July 15, 2011....","{'funny': '', 'posted': 'Posted April 21, 2011...",None,None,None,None,None,None,None
1,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted June 24, 2014....","{'funny': '', 'posted': 'Posted September 8, 2...","{'funny': '', 'posted': 'Posted November 29, 2...",None,None,None,None,None,None,None
2,evcentric,http://steamcommunity.com/id/evcentric,"{'funny': '', 'posted': 'Posted February 3.', ...","{'funny': '', 'posted': 'Posted December 4, 20...","{'funny': '', 'posted': 'Posted November 3, 20...","{'funny': '', 'posted': 'Posted October 15, 20...","{'funny': '', 'posted': 'Posted October 15, 20...","{'funny': '', 'posted': 'Posted October 15, 20...",None,None,None,None
3,doctr,http://steamcommunity.com/id/doctr,"{'funny': '', 'posted': 'Posted October 14, 20...","{'funny': '', 'posted': 'Posted July 28, 2012....","{'funny': '', 'posted': 'Posted June 2, 2012.'...","{'funny': '', 'posted': 'Posted June 29, 2014....","{'funny': '', 'posted': 'Posted November 22, 2...","{'funny': '', 'posted': 'Posted February 23, 2...",None,None,None,None
4,maplemage,http://steamcommunity.com/id/maplemage,"{'funny': '3 people found this review funny', ...","{'funny': '1 person found this review funny', ...","{'funny': '2 people found this review funny', ...","{'funny': '', 'posted': 'Posted July 11, 2013....",None,None,None,None,None,None


Ahora utilizaremos una funcion de pandas llamada melt para poder tener <br>
a cada usuario con la informacion de las columnas, es decir; te permite <br>
elegir las columnas que quieres "fundir" o transformar en filas

In [9]:
# Se utiliza melt escogiendo las 9 columnas para fundir y se dejan fijas user id y user url
df_reviews_melt = pd.melt(df_reviews_norm, id_vars=['user_id', 'user_url'], 
                       value_vars=list(range(9)),
                       value_name='reviews')
df_reviews_melt.head()

,user_id,user_url,variable,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,0,"{'funny': '', 'posted': 'Posted November 5, 20..."
1,js41637,http://steamcommunity.com/id/js41637,0,"{'funny': '', 'posted': 'Posted June 24, 2014...."
2,evcentric,http://steamcommunity.com/id/evcentric,0,"{'funny': '', 'posted': 'Posted February 3.', ..."
3,doctr,http://steamcommunity.com/id/doctr,0,"{'funny': '', 'posted': 'Posted October 14, 20..."
4,maplemage,http://steamcommunity.com/id/maplemage,0,"{'funny': '3 people found this review funny', ..."


In [10]:
df_reviews_melt.isna()

,user_id,user_url,variable,reviews
0,False,False,False,False
1,False,False,False,False
2,False,False,False,False
3,False,False,False,False
4,False,False,False,False
...,...,...,...,...
232186,True,True,False,True
232187,True,True,False,True
232188,True,True,False,True
232189,True,True,False,True


Al hacer el Melt es normal que queden reviews que tengan None por la misma transformación <br>
que se realizó, así que se procede a eliminar

In [11]:
#Quitamos los nulos 

df_reviews_melt = df_reviews_melt.dropna()
df_reviews_melt.isna()

,user_id,user_url,variable,reviews
0,False,False,False,False
1,False,False,False,False
2,False,False,False,False
3,False,False,False,False
4,False,False,False,False
...,...,...,...,...
231291,False,False,False,False
231293,False,False,False,False
231419,False,False,False,False
231499,False,False,False,False


Se convierte cada diccionario en columnas con prefijo review

In [12]:
# Se convierte cada diccionario en columnas con prefijo review_ 
df_reviews = df_reviews_melt['reviews'].apply(pd.Series, dtype='object')
df_reviews = df_reviews.add_prefix('reviews_')
df_reviews.head()

,reviews_funny,reviews_posted,reviews_last_edited,reviews_item_id,reviews_helpful,reviews_recommend,reviews_review
0,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
2,,Posted February 3.,,248820,No ratings yet,True,A suitably punishing roguelike platformer. Wi...
3,,"Posted October 14, 2013.",,250320,2 of 2 people (100%) found this review helpful,True,This game... is so fun. The fight sequences ha...
4,3 people found this review funny,"Posted April 15, 2014.",,211420,35 of 43 people (81%) found this review helpful,True,Git gud


Se volvió a perder las dos columnas id y url así que se concatenan

In [13]:
#Concatenamos las columnas id y url
df_reviews = pd.concat([df_reviews_melt[['user_id', 'user_url']], df_reviews], axis=1)
df_reviews.head()

,user_id,user_url,reviews_funny,reviews_posted,reviews_last_edited,reviews_item_id,reviews_helpful,reviews_recommend,reviews_review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,js41637,http://steamcommunity.com/id/js41637,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
2,evcentric,http://steamcommunity.com/id/evcentric,,Posted February 3.,,248820,No ratings yet,True,A suitably punishing roguelike platformer. Wi...
3,doctr,http://steamcommunity.com/id/doctr,,"Posted October 14, 2013.",,250320,2 of 2 people (100%) found this review helpful,True,This game... is so fun. The fight sequences ha...
4,maplemage,http://steamcommunity.com/id/maplemage,3 people found this review funny,"Posted April 15, 2014.",,211420,35 of 43 people (81%) found this review helpful,True,Git gud


Se observa que hay valores vacíos, así que se convierten a nulos para contabilizarlos <br>
y tomar acción 

In [14]:
# Se reemplaza vacios por nulos 
df_reviews.replace('', None, inplace=True)
df_reviews.isna().sum()

user_id                    0
user_url                   0
reviews_funny          49498
reviews_posted             0
reviews_last_edited    51499
reviews_item_id            0
reviews_helpful            0
reviews_recommend          0
reviews_review            30
dtype: int64

In [15]:
# Se eliminan las columnas de reviews que tienen casi 90% de los datos en nulos
df_reviews = df_reviews.drop(columns=['reviews_funny', 'reviews_last_edited'])


In [16]:
# Se eliminan los nulos de review_review que representan un porcentaje muy pequeño
df_reviews = df_reviews.dropna(subset=['reviews_review'])
# Se verifican los nulos
df_reviews.isna().sum()

user_id              0
user_url             0
reviews_posted       0
reviews_item_id      0
reviews_helpful      0
reviews_recommend    0
reviews_review       0
dtype: int64

Ahora se trabajará con las fechas, se revisará el formato y se dejará en un estandar

In [17]:
df_reviews['reviews_posted'].nunique

<bound method IndexOpsMixin.nunique of 0          Posted November 5, 2011.
1             Posted June 24, 2014.
2                Posted February 3.
3          Posted October 14, 2013.
4            Posted April 15, 2014.
                    ...            
231291      Posted August 15, 2014.
231293       Posted August 2, 2014.
231419        Posted July 31, 2015.
231499    Posted December 20, 2015.
231501           Posted February 3.
Name: reviews_posted, Length: 57367, dtype: object>

In [18]:
# Función para extraer fechas
def extract_date(text):
    # Patrón de expresión regular
    pattern = r'Posted (\w+\s\d{1,2},\s\d{4}).'

    match = re.search(pattern, text)
    if match:
        date_str = match.group(1)
        # Intentar convertir la fecha a formato "yyyy-mm-dd"
        try:
            date_obj = datetime.strptime(date_str, "%B %d, %Y")
            formatted_date = date_obj.strftime("%Y-%m-%d")
            return formatted_date
        except ValueError:
            return "No hay datos del posteo"
    return "No hay datos del posteo"

# Aplicar la función de extracción de fechas
df_reviews['reviews_date'] = df_reviews['reviews_posted'].apply(extract_date)
df_reviews['reviews_date'].nunique

<bound method IndexOpsMixin.nunique of 0                      2011-11-05
1                      2014-06-24
2         No hay datos del posteo
3                      2013-10-14
4                      2014-04-15
                   ...           
231291                 2014-08-15
231293                 2014-08-02
231419                 2015-07-31
231499                 2015-12-20
231501    No hay datos del posteo
Name: reviews_date, Length: 57367, dtype: object>

In [19]:
#contamos la cantidad de datos que no tenían el formato correcto
count_no_data = (df_reviews['reviews_date'] == "No hay datos del posteo").sum()
print(count_no_data)

9768


In [20]:
# Se elimina la columna antigua ya que tenemos la nueva formateada
df_reviews = df_reviews.drop('reviews_posted', axis=1)

In [21]:
df_reviews.head()

,user_id,user_url,reviews_item_id,reviews_helpful,reviews_recommend,reviews_review,reviews_date
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,2011-11-05
1,js41637,http://steamcommunity.com/id/js41637,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,2014-06-24
2,evcentric,http://steamcommunity.com/id/evcentric,248820,No ratings yet,True,A suitably punishing roguelike platformer. Wi...,No hay datos del posteo
3,doctr,http://steamcommunity.com/id/doctr,250320,2 of 2 people (100%) found this review helpful,True,This game... is so fun. The fight sequences ha...,2013-10-14
4,maplemage,http://steamcommunity.com/id/maplemage,211420,35 of 43 people (81%) found this review helpful,True,Git gud,2014-04-15


In [28]:
# Revisamos cuantos valores unicos tiene reviews_recommended y su tipo de dato
print(df_reviews['reviews_recommend'].dtype)
print("---"*10)
df_reviews['reviews_recommend'].unique()

bool
------------------------------


array([ True, False])

Para temas de optimización vamos a convertir reviews_recommend para tipo de datos Bool

In [29]:
# se cambia el tipo de dato de string a bool
df_reviews['reviews_recommend'] = df_reviews['reviews_recommend'].astype(bool)
print(df_reviews['reviews_recommend'].dtype)

bool


In [30]:
df_reviews.shape

(57367, 7)

In [32]:
reviews_clean = 'data/user_reviews_clean.csv'
df_reviews.to_csv(reviews_clean, index=False, encoding='utf-8')